First, we import the necessary TOM Toolkit classes:

In [ ]:
from tom_alerts.brokers.mars import MARSQueryForm, MARSBroker
from tom_alerts.models import BrokerQuery

We can instantiate the `MARSQueryForm` class with the desired parameters, and calling `is_valid()` ensures that our datatypes are consistent with what's specified in the form class. As you can see, we end up with a dictionary of our query parameters.

In [ ]:
mars_form = MARSQueryForm({
    'time__since': 864000,
    'objectcone': 'm101, 5',
    'rb__gte': 0.95
})
mars_form.is_valid()
print(mars_form.cleaned_data)

Now, we can go ahead and run the query on MARS. With our `MARSQueryForm` instance, it's as simple as instantiating the `MARSBroker` and calling `fetch_alerts()` on the cleaned data.

In [ ]:
mars = MARSBroker()

alerts = mars.fetch_alerts(mars_form.cleaned_data)

print(sum(1 for _ in alerts))

With the TOM Toolkit, we can also store our query in our TOM as a `BrokerQuery` object with just a few parameters--a name for the query, the name of the `MARSBroker`, and the query parameters.

In [ ]:
query = BrokerQuery.objects.create(
    name='Test Query',
    broker=mars.name,
    parameters=mars_form.serialize_parameters()
)

It's now trivial to simply get the query out of the database, and run it again on MARS:

In [ ]:
query = BrokerQuery.objects.filter(name='Test Query', broker=mars.name)[0]

alerts = mars.fetch_alerts(query.parameters_as_dict)
print(sum(1 for _ in alerts))

With the available alerts from a query, we can convert each of them to a TOM `Target` object and store them in our local TOM DB.

In [ ]:
alerts = mars.fetch_alerts(query.parameters_as_dict)
for alert in alerts:
    mars.to_target(alert)

And as expected, we now have as many `Target` objects in our TOM as we had alerts, which can be used for later reference.

In [ ]:
from tom_targets.models import Target

print(len(Target.objects.all()))